## Importing the libraries

In [60]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
INPUT_FILE = 'summary_input.csv'
OUTPUT_FILE = 'summary_result.csv'
TRANSACTION_COLUMN = 'Transaction'
CATEGORY_COLUMN = 'Category'
LABEL_COLUMN = 'Label'

### Import Dataset

In [62]:
# Just make sure the columns 'Transaction' and 'Category' exist. Otherwise you need to change the code
dataset = pd.read_csv(INPUT_FILE)

print(dataset.head())

                     Transaction        Category
0      PETROCAN-2100 BURNHAMTHOR  TRANSPORTATION
1         TOMO SUSHI MISSISSAUGA            FOOD
2         TOMO SUSHI MISSISSAUGA            FOOD
3  PIZZA PIZZA # 266 MISSISSAUGA            FOOD
4        BAR BURRITO MISSISSAUGA            FOOD


# Preprocessing

### Data cleaning

In [63]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Transaction  100 non-null    object
 1   Category     99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [64]:
# find out is there is any null values
dataset.isna().sum()

Transaction    0
Category       1
dtype: int64

In [65]:
# if there is any null values, the call below will elminate those rows
dataset = dataset.dropna()

In [66]:
# check after cleaning for null values
dataset.isna().sum()

Transaction    0
Category       0
dtype: int64

In [67]:
# Some transactions which only 1 unique count may not be enough for the model to learn
# add more rows (a least 2 more) for those unique values to the data ...
item_counts = dataset[TRANSACTION_COLUMN].value_counts()
print(item_counts.sort_values(ascending=True))

# Count unique values in column 'Transaction'
value_counts_trans = dataset[TRANSACTION_COLUMN].value_counts()

# Filter DataFrame based on the count of unique values in column 'Transaction'
filtered_df = dataset[dataset[TRANSACTION_COLUMN].isin(value_counts_trans.index[value_counts_trans < 2])]

# print(filtered_df)

Transaction
ANNUAL FEE                        1
SUNOCO 0442552600 CANANDAIGUA     1
PIZZA PIZZA # 266 MISSISSAUGA     1
SHELL C20130 MISSISSAUGA          1
TIM HORTONS #6010 TORONTO         1
PETROCAN-1405 SOUTHDOWN R         1
SITEPOINT WWW.SITEPOIN            1
Subway 33041 Mississauga          1
EXXON SUN FARMINGTON              1
PETROCAN-3990 EGLINTON AV         1
NEW WOK NEW WOK TORONTO           1
CASH ADVANCE FEE                  1
PETRO CANADA00453 MISSISSAUGA     1
GLEN ERIN ESSO MISSISSAUGA        1
SYSTEM76 Y7Q45 SYSTEM76.COM       1
PETRO CANADA05618 MISSISSAUGA     1
RETAIL INTEREST                   1
MOBIL@ - 4372 MISSISSAUGA         1
MIFUEL #878 MISSISSAUGA           1
AMZN Mktp CA*AN5XM39E3            1
ENBRIDGE GAS H6Z9A7               1
APPLE.COM/CA 800-676-2775         1
TIM HORTONS #5795 TORONTO         1
ENERSOURCE   L6Y9Q2               1
TIM HORTONS #1431 OAKVILLE        2
SUBARU FINANCE                    2
CASH INTEREST                     2
PETRO CANADA0047

In [68]:
# remove space in column values
import re

dataset[TRANSACTION_COLUMN] = dataset[TRANSACTION_COLUMN].str.strip().replace('\s+', ' ', regex=True)
dataset[CATEGORY_COLUMN] = dataset[CATEGORY_COLUMN].str.strip().replace('\s+', ' ', regex=True)

print(dataset)


                      Transaction        Category
0       PETROCAN-2100 BURNHAMTHOR  TRANSPORTATION
1          TOMO SUSHI MISSISSAUGA            FOOD
2          TOMO SUSHI MISSISSAUGA            FOOD
3   PIZZA PIZZA # 266 MISSISSAUGA            FOOD
4         BAR BURRITO MISSISSAUGA            FOOD
..                            ...             ...
95              ENERSOURCE L6Y9Q2         Utility
96            ENBRIDGE GAS H6Z9A7         Utility
97                 SUBARU FINANCE      Rent/Lease
98                 SUBARU FINANCE      Rent/Lease
99                   MANULIFE MSP  Life Insurance

[99 rows x 2 columns]


In [69]:
dataset[CATEGORY_COLUMN].unique()

array(['TRANSPORTATION', 'FOOD', 'CELL PHONE', 'OFFICE', 'INTERNET',
       'EDUCATION', 'BANK FEE', 'Life Insurance', 'Utility', 'Rent/Lease'],
      dtype=object)

### Label encoding

In [70]:
# Label encoding and create a new column with the encoded data
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(dataset[CATEGORY_COLUMN])
# create new column with the encoded values
dataset[LABEL_COLUMN] = label_encoder.transform(dataset[CATEGORY_COLUMN])

In [71]:
dataset.tail()

,Transaction,Category,Label
95,ENERSOURCE L6Y9Q2,Utility,9
96,ENBRIDGE GAS H6Z9A7,Utility,9
97,SUBARU FINANCE,Rent/Lease,7
98,SUBARU FINANCE,Rent/Lease,7
99,MANULIFE MSP,Life Insurance,5


### Creating the feature and dependent variables

In [72]:
X = dataset[TRANSACTION_COLUMN]
y = dataset[LABEL_COLUMN]

### Creating the Bag of Words model

In [73]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
vectorized_X = vectorizer.fit_transform(X)

### Split training and test sets

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectorized_X, y, test_size = 0.20, random_state = 0)

In [75]:
print(X_train)
print()
print(y_train)

  (0, 7)	0.4472135954999579
  (0, 19)	0.4472135954999579
  (0, 16)	0.4472135954999579
  (0, 63)	0.4472135954999579
  (0, 52)	0.4472135954999579
  (1, 60)	0.42854038388020804
  (1, 84)	0.6388869772439425
  (1, 88)	0.6388869772439425
  (2, 5)	0.4411912904654138
  (2, 73)	0.8823825809308276
  (2, 60)	0.1635580205533022
  (3, 86)	0.5773502691896257
  (3, 65)	0.5773502691896257
  (3, 41)	0.5773502691896257
  (4, 60)	0.42854038388020804
  (4, 84)	0.6388869772439425
  (4, 88)	0.6388869772439425
  (5, 74)	0.6087610104583493
  (5, 10)	0.6087610104583493
  (5, 71)	0.5087436135141739
  (6, 74)	0.6087610104583493
  (6, 10)	0.6087610104583493
  (6, 71)	0.5087436135141739
  (7, 66)	0.7071067811865476
  (7, 58)	0.7071067811865476
  :	:
  (72, 3)	0.5895433653453508
  (72, 56)	0.5259303476241797
  (72, 87)	0.5259303476241797
  (72, 60)	0.31501295096090465
  (73, 89)	0.5427906691887742
  (73, 13)	0.6319693598164365
  (73, 56)	0.39114768163368047
  (73, 87)	0.39114768163368047
  (74, 55)	0.57735026918962

### Training the model

In [76]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

# from xgboost import XGBClassifier
# classifier = XGBClassifier(objective="multi:softmax", num_class=3)
# classifier.fit(X_train, y_train)

RandomForestClassifier()

### Prediction and classification

In [77]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2 0 0 0 0 0 0]
 [0 3 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 9 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 3 0]
 [0 0 0 0 0 1 0]]


0.95

As above show the accuraccy is above 80% (for my data of course). No bad. In the future will use different models to see if there is an improvement in accuracy

### Applying the model

In [78]:
# I remove the Date, amount columns for privacy. Don't do that with real data
dataset_result = pd.read_csv(OUTPUT_FILE)

# removing 'Category' column if exist. It will be added later by the prediction ...
if CATEGORY_COLUMN in dataset_result.columns:
    dataset_result.drop(columns=[CATEGORY_COLUMN], inplace=True)

dataset_result = dataset_result.dropna()
print(dataset_result.tail())

             Transaction
89            ANNUAL FEE
90  MANULIFE         MSP
91  MANULIFE         MSP
92  ENBRIDGE GAS H6Z9A7 
93  ENBRIDGE GAS J7W3W7 


Notice above output showing that there is only one column 'Transaction' (Remove 'Category' column if exist).
We are going to add a new column 'Category' that the model is going set the value

In [79]:
# vectorize the data
features = vectorizer.transform(dataset_result[TRANSACTION_COLUMN])

# apply model to get predictions ...
predictions = classifier.predict(features)

# convert output labels to new 'Category' column
dataset_result[CATEGORY_COLUMN] = label_encoder.inverse_transform(predictions)

dataset_result.to_csv(OUTPUT_FILE, index=False)

### Final result ...

In [80]:
print(dataset_result.tail())

             Transaction        Category
89            ANNUAL FEE        BANK FEE
90  MANULIFE         MSP  Life Insurance
91  MANULIFE         MSP  Life Insurance
92  ENBRIDGE GAS H6Z9A7          Utility
93  ENBRIDGE GAS J7W3W7          Utility


### The resulting file summary_result.csv is bang on. No more manual creation of Category. Teach the computer to do it for you.

### Enjoy Machine Learning